In [2]:
import os
import sys
import string
import numpy as np
import pandas as pd
from tensorflow import keras
from tqdm import tqdm
import pickle

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
df = pd.read_csv("Data/musicoset_songfeatures/lyrics.csv", sep='\t')

In [5]:
df.head()

,song_id,lyrics
0,3e9HZxeyfWwjeyPAMmWSSQ,['[Verse 1]\nThought I\'d end up with Sean\nBu...
1,5p7ujcrUXASCNwRaWNHR1C,"[""[Verse 1]\nFound you when your heart was bro..."
2,2xLMifQCjDGFmkHkpNLD9h,"['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun..."
3,3KkXRkHbMCARz0aVfEt68P,NaN
4,1rqqCSm0Qe4I9rUvWncaom,"[""[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t..."


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20404 entries, 0 to 20403
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   song_id  20404 non-null  object
 1   lyrics   19663 non-null  object
dtypes: object(2)
memory usage: 318.9+ KB


In [7]:
df.isna().sum()

song_id      0
lyrics     741
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.reset_index(inplace=True)

In [10]:
df.drop(['index'], axis=1, inplace=True)

In [11]:
print(df.lyrics,sep='\\n\\n')

0        ['[Verse 1]\nThought I\'d end up with Sean\nBu...
1        ["[Verse 1]\nFound you when your heart was bro...
2        ['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun...
3        ["[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t...
4        ["[Intro]\nI-I-I don't want a lot for Christma...
                               ...                        
19658    ['[Verse 1: Big Boi]\nWell, it\'s the M-I-croo...
19659    ['[Intro]\nThere are times when I look above a...
19660    ["[Intro: Prodigy and Havoc]\nWord up son, wor...
19661    ["[Chorus]\nWee-ooh wim-o-weh. Wee-ooh wim-o-w...
19662    ['[Intro: Shaq]\nYo Jef, why don\'t you give m...
Name: lyrics, Length: 19663, dtype: object


* we need to remove punctuations and '[]' in this text

### Preprocessing examples that we need to do
* **1. Remove Punctuations**

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Define traslator

In [13]:
translator= str.maketrans('','', string.punctuation)

In [14]:
# example for removing punchuations
xyz="next\ Thank you,@ next\ Thank you. next Yeah, yee'"
xyz.translate(translator)

'next Thank you next Thank you next Yeah yee'

note:- see above text all punctuations are removed

* **split texts verses, Pre-chorus, and into**

In [15]:
xyz= df.lyrics[1].split('\\n\\n')
xyz[:2]

['["[Verse 1]\\nFound you when your heart was broke\\nI filled your cup until it overflowed\\nTook it so far to keep you close (Keep you close)\\nI was afraid to leave you on your own',
 "[Pre-Chorus]\\nI said I'd catch you if you fall (Fall)\\nAnd if they laugh, then fuck 'em all (All)\\nAnd then I got you off your knees\\nPut you right back on your feet\\nJust so you could take advantage of me"]

In [16]:
key= xyz[0][xyz[0].find('[') + 1:xyz[0].find(']')].strip()
key

'"[Verse 1'

we have intro square brackets which have subheading infront of them we need to drop only intro word

In [17]:
xyz= df.lyrics[19662].split('\\n\\n')
xyz[:2]

["['[Intro: Shaq]\\nYo Jef, why don\\'t you give me a hoopa beat or something\\nSomething I can go to the park to\\nYeah, there you go, allright, I like that, I like that\\nIt sound dope",
 "(Bust \\'em in the eye Shaq)"]

In [18]:
key= xyz[0][xyz[0].find('[')+1:  xyz[0].find(']')].strip()
key

"'[Intro: Shaq"

In [19]:
if ':' in key:
#     take evrything till ':' comes
    key= key[:key.find(':')]
    
key  

"'[Intro"

In [20]:
# take exrything after square brackets end
# need +1 beause skip ']' <--this
xyz[0][xyz[0].find(']')+1:]

"\\nYo Jef, why don\\'t you give me a hoopa beat or something\\nSomething I can go to the park to\\nYeah, there you go, allright, I like that, I like that\\nIt sound dope"

# Clean this text as all above examples

In [21]:
df.head()

,song_id,lyrics
0,3e9HZxeyfWwjeyPAMmWSSQ,['[Verse 1]\nThought I\'d end up with Sean\nBu...
1,5p7ujcrUXASCNwRaWNHR1C,"[""[Verse 1]\nFound you when your heart was bro..."
2,2xLMifQCjDGFmkHkpNLD9h,"['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun..."
3,1rqqCSm0Qe4I9rUvWncaom,"[""[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t..."
4,0bYg9bo50gSsH3LtXe2SQn,"[""[Intro]\nI-I-I don't want a lot for Christma..."


In [22]:
df.lyrics

0        ['[Verse 1]\nThought I\'d end up with Sean\nBu...
1        ["[Verse 1]\nFound you when your heart was bro...
2        ['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun...
3        ["[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t...
4        ["[Intro]\nI-I-I don't want a lot for Christma...
                               ...                        
19658    ['[Verse 1: Big Boi]\nWell, it\'s the M-I-croo...
19659    ['[Intro]\nThere are times when I look above a...
19660    ["[Intro: Prodigy and Havoc]\nWord up son, wor...
19661    ["[Chorus]\nWee-ooh wim-o-weh. Wee-ooh wim-o-w...
19662    ['[Intro: Shaq]\nYo Jef, why don\'t you give m...
Name: lyrics, Length: 19663, dtype: object

In [23]:
text=[]

In [24]:
for i in df.lyrics:
    for verse in i.split("\\n\\n"):
        text.append(verse)

In [25]:
replace_list =  ['[Verse 1]','[Verse 2]','[Verse 3]','[Verse 4]', '[Chorus]']

In [26]:
text[0]

'[\'[Verse 1]\\nThought I\\\'d end up with Sean\\nBut he wasn\\\'t a match\\nWrote some songs about Ricky\\nNow I listen and laugh\\nEven almost got married\\nAnd for Pete, I\\\'m so thankful\\nWish I could say, "Thank you" to Malcolm\\n\\\'Cause he was an angel'

In [27]:
cleaned_text= []

In [28]:
for s in tqdm(text):
   for i in replace_list:
     if i in s:
       cleaned_text += [x.lower().replace('(','').replace(')','').translate(translator) for x in s[s.find(']')+1:].split('\\n') if len(x) > 1]


100%|██████████████████████████████████████████████████████████████████████| 486430/486430 [00:01<00:00, 428856.01it/s]


In [29]:
len(cleaned_text)


237764

In [30]:
text[0:5]

['[\'[Verse 1]\\nThought I\\\'d end up with Sean\\nBut he wasn\\\'t a match\\nWrote some songs about Ricky\\nNow I listen and laugh\\nEven almost got married\\nAnd for Pete, I\\\'m so thankful\\nWish I could say, "Thank you" to Malcolm\\n\\\'Cause he was an angel',
 "[Pre-Chorus]\\nOne taught me love\\nOne taught me patience\\nAnd one taught me pain\\nNow, I\\'m so amazing\\nSay I\\'ve loved and I\\'ve lost\\nBut that\\'s not what I see\\nSo, look what I got\\nLook what you taught me\\nAnd for that, I say",
 "[Chorus]\\nThank you, next (Next)\\nThank you, next (Next)\\nThank you, next\\nI\\'m so fuckin\\' grateful for my ex\\nThank you, next (Next)\\nThank you, next (Next)\\nThank you, next (Next)\\nI\\'m so fuckin\\'—",
 "[Verse 2]\\nSpend more time with my friends\\nI ain\\'t worried \\'bout nothin\\'\\nPlus, I met someone else\\nWe havin\\' better discussions\\nI know they say I move on too fast\\nBut this one gon\\' last\\n\\'Cause her name is Ari\\nAnd I\\'m so good with that (So 

In [31]:
cleaned_text[:10]

['thought id end up with sean',
 'but he wasnt a match',
 'wrote some songs about ricky',
 'now i listen and laugh',
 'even almost got married',
 'and for pete im so thankful',
 'wish i could say thank you to malcolm',
 'cause he was an angel',
 'thank you next next',
 'thank you next next']

**Note**- this cleaned_text file will be needed in GPT-2 training so save it

In [32]:
with open("data/cleaned_text.txt", 'wb') as fp:
    pickle.dump(cleaned_text, fp)

# prep


In [59]:
text_corpus =  []
for sentence in cleaned_text:
    for word in sentence.split():
        text_corpus.append(word)

In [61]:
text_corpus[:5]

['thought', 'id', 'end', 'up', 'with']

In [74]:
MIN_SEQ = 5
MIN_FREQUENCY=7

In [63]:
frequency= {}
uncommon_words= set()


In [64]:
for w in text_corpus:
    frequency[w] = frequency.get(w,0)+1

In [83]:
# Take uncommon words means if frequency of word is less than 7 take it as uncomman word
uncommon_words = set([key for key in frequency.keys() if frequency[key] < MIN_FREQUENCY])
# take all words who are not in uncommon words
vocab = sorted(set([key for key in frequency.keys() if frequency[key]>= MIN_FREQUENCY]))

In [84]:
VOCAB_SIZE= len(vocab)
VOCAB_SIZE

7233

In [ ]:
### Create dictionary of words to their indices from ours vocabulary i.e. words

In [85]:
word_indices = dict((w,i) for i ,w in enumerate(vocab))
indices_word= dict((i,w) for i, w in enumerate(vocab))

In [75]:
sequences= []
next_word= []

In [76]:
text_corpus[5:10]

['sean', 'but', 'he', 'wasnt', 'a']

### **Imp Note** - here we will create unique data set of lstm with 5 words as input and next word after than 5 will be our prediction/label

In [77]:
for i in tqdm(range(len(text_corpus)- MIN_SEQ)):
    if text_corpus[i] in vocab:
        sequences.append(text_corpus[i:i+MIN_SEQ])
        next_word.append(text_corpus[i+MIN_SEQ])

100%|█████████████████████████████████████████████████████████████████████| 1662102/1662102 [01:43<00:00, 16059.07it/s]


In [79]:
sequences[:5]

[['thought', 'id', 'end', 'up', 'with'],
 ['id', 'end', 'up', 'with', 'sean'],
 ['end', 'up', 'with', 'sean', 'but'],
 ['up', 'with', 'sean', 'but', 'he'],
 ['with', 'sean', 'but', 'he', 'wasnt']]

In [81]:
next_word[:5]

['sean', 'but', 'he', 'wasnt', 'a']

**note save this word_indicies, indicies_words will need them at time of creating vectors from words**

In [86]:
with open('data/word_indices.txt','wb') as fp:
    pickle.dump(word_indices, fp)

In [87]:
with open('data/indices_word.txt','wb') as fp:
    pickle.dump(indices_word, fp)

### Save sequences and next_word list we will use them at time of crating batch

In [91]:
# save 
with open("Data/sequences.txt", 'wb') as fp:
  pickle.dump(sequences, fp)

In [92]:
# load
with open("Data/sequences.txt", 'rb') as fp:
  loaded_x= pickle.load(fp)

In [93]:
# check if previous and loaded is same
loaded_x == sequences

True

In [94]:
with open("data/next_word.txt", 'wb') as fp:
  pickle.dump(next_word, fp)

In [95]:
with open("Data/next_word.txt", 'rb') as fp:
  loaded_y= pickle.load(fp)

In [97]:
loaded_y == next_word

True